In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("sample.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
docs=text_splitter.split_documents(documents)

Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 242, which is longer than the specified 200
Created a chunk of size 270, which is longer than the specified 200


In [16]:
docs

[Document(metadata={'source': 'sample.txt'}, page_content='In Germany, on the night of November 9, 1938, a quiet dread filled the air. The streets began to fill with the sounds of shattering glass, fire, and screams. That night, and into the next day, nearly 7,500 Jewish-owned businesses were smashed, hundreds of synagogues were burned, and thousands of Jewish men were arrested and sent to concentration camps.'),
 Document(metadata={'source': 'sample.txt'}, page_content='What triggered this violent outbreak?'),
 Document(metadata={'source': 'sample.txt'}, page_content='A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'),
 Document(metadata={'source': 'sample.txt'}, page_content='The event was deceptively called Kristallnacht, or "The Night of Broken Glass" â€” referencing the glass from all the s

In [17]:
embeddings = OllamaEmbeddings(model="gemma:2b")

db = FAISS.from_documents(docs, embeddings)
db

In [19]:
query = "What was the main motive of Herschel?"
docs = db.similarity_search(query)
docs[0].page_content

'A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'

In [21]:
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'

In [22]:
docs_and_score = db.similarity_search_with_score(query)
docs_and_score

[(Document(id='85652148-01f2-4f5f-aa65-9abc456cdfbc', metadata={'source': 'sample.txt'}, page_content='A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'),
  np.float32(2680.3208)),
 (Document(id='e6a2f1b0-2430-4efa-b367-71ea78440763', metadata={'source': 'sample.txt'}, page_content='What triggered this violent outbreak?'),
  np.float32(2976.0054)),
 (Document(id='b27d9e76-4d2f-4b24-b751-203184dc2c40', metadata={'source': 'sample.txt'}, page_content="After Kristallnacht, there was no doubt: the Nazi regime's goal was not discriminationâ€”it was destruction. This moment became a turning point in the Holocaust, foreshadowing the horrors to come."),
  np.float32(3295.3755)),
 (Document(id='6571edfa-5cc3-4e56-b533-8ed1f621e226', metadata={'source': 'sample.txt'}, page_content='The event was deceptive

In [23]:
embeddings_vector = embeddings.embed_query(query)
embeddings_vector

[-0.5206849575042725,
 -0.8223971724510193,
 -0.19833092391490936,
 2.9403843879699707,
 1.1524269580841064,
 1.494686484336853,
 0.5985803604125977,
 0.5162062048912048,
 0.03412343189120293,
 0.08778475224971771,
 -0.610451877117157,
 -0.7000858187675476,
 -1.0817549228668213,
 0.8055028319358826,
 -0.5236240029335022,
 -0.7911686897277832,
 5.32456111907959,
 -0.26151102781295776,
 0.8342505693435669,
 -0.02970302477478981,
 1.1659210920333862,
 -0.05776781216263771,
 -0.11980243027210236,
 -0.7316868901252747,
 -2.0567755699157715,
 -0.03515142574906349,
 1.4716925621032715,
 -0.7391271591186523,
 -0.6365041732788086,
 -1.692470669746399,
 -0.5138774514198303,
 -0.4933619499206543,
 -0.27467069029808044,
 -0.06616772711277008,
 -0.358700692653656,
 0.3943435549736023,
 0.12482095509767532,
 -1.4204483032226562,
 0.7422980070114136,
 -1.5636258125305176,
 0.6449238657951355,
 -0.1535104215145111,
 0.7009891271591187,
 -2.0297958850860596,
 1.2343437671661377,
 0.4038298428058624,
 0

In [24]:
docs_score = db.similarity_search_by_vector(embeddings_vector)
docs_score

[Document(id='85652148-01f2-4f5f-aa65-9abc456cdfbc', metadata={'source': 'sample.txt'}, page_content='A 17-year-old Jewish boy named Herschel Grynszpan, upset by the Nazi deportation of his family, had shot a German diplomat in Paris. The Nazi leadership used it as a pretext to unleash a coordinated, nationwide attack on the Jewish community.'),
 Document(id='e6a2f1b0-2430-4efa-b367-71ea78440763', metadata={'source': 'sample.txt'}, page_content='What triggered this violent outbreak?'),
 Document(id='b27d9e76-4d2f-4b24-b751-203184dc2c40', metadata={'source': 'sample.txt'}, page_content="After Kristallnacht, there was no doubt: the Nazi regime's goal was not discriminationâ€”it was destruction. This moment became a turning point in the Holocaust, foreshadowing the horrors to come."),
 Document(id='6571edfa-5cc3-4e56-b533-8ed1f621e226', metadata={'source': 'sample.txt'}, page_content='The event was deceptively called Kristallnacht, or "The Night of Broken Glass" â€” referencing the glass 

In [25]:
db.save_local("faiss_index")

In [29]:
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query)